# Tools for a Customer Outreach Campaign

We have already discussed the importance of these in readme.md on main page:
- Versatility
- Fault Tolerance
- Caching

In [39]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [40]:
from crewai import Agent, Task, Crew

In [41]:
import warnings
warnings.filterwarnings('ignore')

For better results, we can use `gpt-4-turbo`, but since this is just a showcase, I will use a `gpt-3.5-turbo`.

In [42]:
import os
#from utils import get_openai_api_key

from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = serper_api_key

## Creating Agents

In [43]:
sales_rep_agent = Agent(
    role = "Sales Representative",
    goal = "Identify high value leads that match our ideal customer profile",
    backstory = ("As a part of dynamic sales team at HuggingFace,"
                 "your mission is to scour the digital landscape for potential leads."
                 "Armed with cutting-edge tools and strategic mindset, you analyze the data,"
                 "trends and interactions to unearth opportunities that others might overlook."
                 "Your work is crucial in paving the way for meaningful engagement and driving the company's growth."
                 ),
    allow_delegation = False,
    verbose = True
)

In [44]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of HuggingFace's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

### Creating Tools


In [62]:
"""
!pip install duckduckgo-search -U
from langchain_community.tools import DuckDuckGoSearchRun
"""

  Obtaining dependency information for duckduckgo-search from https://files.pythonhosted.org/packages/3b/4d/2aebe0e326ec8c25a6709c52217367a02db202854b048f5df961b679ea45/duckduckgo_search-6.1.7-py3-none-any.whl.metadata
  Obtaining dependency information for pyreqwest-impersonate>=0.4.8 from https://files.pythonhosted.org/packages/fc/f2/242011b6940be7efda73e03b36672fdd4422cb678528dd235440dbe08e5a/pyreqwest_impersonate-0.4.8-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for orjson>=3.10.5 from https://files.pythonhosted.org/packages/5b/0d/96079bcd0d7f6cda1719c1cbb07bb136049123f46b70be085610f625f6cd/orjson-3.10.5-cp311-cp311-macosx_10_15_x86_64.macosx_11_0_arm64.macosx_10_15_universal2.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.7/258.7 kB 5.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.9 MB/s eta 0:00:00a 0:00:01
  Attempting

In [63]:
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool

directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()
#search_tool = DuckDuckGoSearchRun()

Lets create our own custom tools for sentiment analysis too. 

Every Tool needs to have 
- a `name` and
- a `description`.
For simplicity lets define this tool, `SentimentAnalysisTool` to return positive for every text.

In [47]:
from crewai_tools import BaseTool

class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Our custom code tool goes here
        return "positive"
    
sentiment_analysis_tool = SentimentAnalysisTool()

### Creating Tasks

In [48]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

the Personalized Outreach Task will use our custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [49]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

### Creating and running the crew

In [50]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

2024-06-20 14:43:12,373 - 8052547712 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [51]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather as much information as possible about DeepLearningAI to create a comprehensive report.

Action: List files in directory
Action Input: {} 

File paths: 
-



Thought: I should start by reading the content of the available files to gather information about DeepLearningAI.

Action: Read a file's content
Action Input: {"file_path": "./instructions/DeepLearningAI_profile.txt"} 

I encountered 

2024-06-20 14:43:54,488 - 8052547712 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--ae4dc4b501da28f3a90a30d6d85da7242999fed25b550e00c9bf0bc56925644b


Thought: I need to gather information about DeepLearningAI from alternative sources since I couldn't access the file provided.

Action: duckduckgo_search
Action Input: {"query": "DeepLearningAI company profile"}
 

I encountered an error while trying to use the tool. This was the error: https://links.duckduckgo.com/d.js?q=DeepLearningAI+company+profile&kl=wt-wt&l=wt-wt&p=&s=0&df=y&vqd=4-253746648595863132247718852483561846540&ex=-1 202 Ratelimit.
 Tool duckduckgo_search accepts these inputs: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.



2024-06-20 14:44:02,621 - 8052547712 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--fd0a4845045bfb1fda11e6e15aac4109e1aeea1a6194b7dfb7308a3df7b1a63a


Thought: I need to find an alternative way to gather information about DeepLearningAI, as the initial tool did not work.

Action: duckduckgo_search
Action Input: {"query": "DeepLearningAI company information"}
 

I encountered an error while trying to use the tool. This was the error: https://links.duckduckgo.com/d.js?q=DeepLearningAI+company+information&kl=wt-wt&l=wt-wt&p=&s=0&df=y&vqd=4-117256830818778976775005292390797868813&ex=-1 202 Ratelimit.
 Tool duckduckgo_search accepts these inputs: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.



2024-06-20 14:44:13,720 - 8052547712 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--ae4dc4b501da28f3a90a30d6d85da7242999fed25b550e00c9bf0bc56925644b


Thought: I need to gather information about DeepLearningAI from alternative sources since I couldn't access the file provided.

Action: duckduckgo_search
Action Input: {"query": "DeepLearningAI company profile"}
 

I encountered an error while trying to use the tool. This was the error: https://links.duckduckgo.com/d.js?q=DeepLearningAI+company+profile&kl=wt-wt&l=wt-wt&p=&s=0&df=y&vqd=4-253746648595863132247718852483561846540&ex=-1 202 Ratelimit.
 Tool duckduckgo_search accepts these inputs: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.

Final Answer:

Subject: Elevating DeepLearningAI's Recent Product Launch with HuggingFace Solutions

Dear Andrew Ng,

I hope this message finds you well. I am reaching out to you from HuggingFace, a cutting-edge AI company that admires DeepLearningAI's innovative strides in the Online Learning Platform sector.

Despite the limitations in accessing detailed informat

In [53]:
from IPython.display import Markdown
Markdown(result)

Subject: Elevating DeepLearningAI's Recent Product Launch with HuggingFace Solutions

Dear Andrew Ng,

I hope this message finds you well. I am reaching out to you from HuggingFace, a cutting-edge AI company that admires DeepLearningAI's innovative strides in the Online Learning Platform sector.

Despite the limitations in accessing detailed information about your recent product launch, our team recognizes the value and impact of your advancements in the industry. At HuggingFace, we specialize in providing tailored AI solutions that align perfectly with the forward-thinking vision of companies like DeepLearningAI.

With a deep commitment to understanding your business needs and objectives, we believe that our technology can complement your recent achievements and propel DeepLearningAI towards even greater success. Our personalized approach ensures that our solutions are not only aligned with your goals but also enhance your existing capabilities.

I am confident that our partnership can result in a synergistic collaboration that unlocks new possibilities for DeepLearningAI. By leveraging our expertise in AI, we can help you optimize processes, drive efficiencies, and unlock untapped potential within your organization.

I would love the opportunity to discuss how HuggingFace can support your future endeavors and contribute to the continued success of DeepLearningAI. Please let me know a convenient time for a meeting or call to explore this further. 

Thank you for considering our proposal, and I look forward to the possibility of working together to achieve mutual success.

Warm regards,

[Your Name]
Lead Sales Representative
HuggingFace

Note: Please ensure to personalize the email further with your contact information and any additional details that may strengthen the connection with Andrew Ng.